# Ringland Festival kaart - GPX data

In [13]:
import pandas as pd
import geopandas
from shapely.geometry import Point

In [5]:
df = pd.read_csv('../curieuzeneuzen_20160613.csv', sep=';')

In [9]:
df['no2'] = np.random.uniform(20,80, size=len(df))

In [60]:
gdf = geopandas.GeoDataFrame(df[['no2', 'Returned']], geometry=df[['Lon', 'Lat', 'no2']].apply(lambda x: Point(x['Lon'], x['Lat']), axis=1))

In [56]:
def to_gpx(gdf, fpath=None):
    """
    Only for Points for now
    """
    
    import gpxpy.gpx
    
    geoms = gdf.geometry
    
    gpx = gpxpy.gpx.GPX()
    wps = []
    
    for p in geoms:
        wps.append(gpxpy.gpx.GPXWaypoint(latitude=p.y, longitude=p.x))
    
    gpx.waypoints.extend(wps)
    
    if fpath is None:
        return gpx.to_xml()
    else:
        with open(fpath, 'w') as f:
            f.write(gpx.to_xml())

In [62]:
subset = gdf[gdf['Returned']==1]

In [63]:
to_gpx(subset, 'test2.gpx')